In [0]:
weather_df = spark.table('`my-side-projects`.bronze_nyc_taxis.bronze_weather_oct_dec_2024')
taxis_df = spark.table('`my-side-projects`.bronze_nyc_taxis.bronze_nyc_taxis_oct_dec_2024')
lookup_df = spark.table('`my-side-projects`.bronze_nyc_taxis.bronze_taxi_zone_lookup')

In [0]:
weather_df.show(1)

In [0]:
from pyspark import broadcast
from pyspark.sql.functions import col, date_trunc

#Using broadcast join because location table contains ~250 records so it is small enough to be cached and joined
# loc_dict = broadcast(lookup_df)
# weather_brd = broadcast(weather_df)

#Truncating pickup time to enable joining weather data (weather data available for full hours)
dt_taxis_hour = taxis_df.withColumn("pickup_hour", date_trunc("hour", col("tpep_pickup_datetime")))

#Using broadcast join because weather table contains ~720 records so it is small enough to be cached and joined
dt_taxis_weather = dt_taxis_hour.join(weather_df, dt_taxis_hour.pickup_hour == weather_df.datetime, "left").drop(col("tpep_pickup_datetime"), col("tpep_dropoff_datetime"), col("datetime"))

#Joining dictionary table to create final result table ready to be analysed
result_table = dt_taxis_weather.join(lookup_df.withColumnRenamed("LocationID", "PULocationID_tmp")
                                    .withColumnRenamed("Borough", "PUBorough").drop(col("service_zone"))
                                    .withColumnRenamed("Zone", "PUZone"),dt_taxis_weather.PULocationID == col("PULocationID_tmp"),"left").join(lookup_df.withColumnRenamed("LocationID", "DOLocationID_tmp")
                                    .withColumnRenamed("Borough", "DOBorough")
                                    .withColumnRenamed("Zone", "DOZone"),dt_taxis_weather.DOLocationID == col("DOLocationID_tmp"),"left").drop("PULocationID_tmp", "DOLocationID_tmp")
